In [33]:
!nvidia-smi


Fri Jan 30 18:59:46 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   39C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [34]:
%%writefile cpu_openmp.cpp
#include <iostream>                     // Я подключаю ввод-вывод
#include <vector>                       // Я подключаю vector для массива
#include <omp.h>                        // Я подключаю OpenMP для параллельности на CPU
#include <chrono>                       // Я подключаю chrono для замера времени

using namespace std;                    // Я использую стандартное пространство имён

int main() {                            // Я начинаю основную программу
    const int N = 1000000;              // Я задаю размер массива (1 000 000)
    vector<float> data(N, 1.0f);        // Я создаю массив и заполняю его единицами

    auto start = chrono::high_resolution_clock::now();  // Я фиксирую старт времени

    #pragma omp parallel for            // Я распараллеливаю цикл по потокам CPU
    for (int i = 0; i < N; i++) {       // Я прохожусь по всем элементам массива
        data[i] *= 2.0f;                // Я умножаю каждый элемент на 2
    }                                   // Я заканчиваю цикл

    auto end = chrono::high_resolution_clock::now();    // Я фиксирую конец времени
    chrono::duration<double, milli> ms = end - start;   // Я считаю длительность в миллисекундах

    cout << "CPU OpenMP time: " << ms.count() << " ms" << endl; // Я вывожу время

    cout << "CPU check (first 5): ";    // Я вывожу проверку первых 5 элементов
    for (int i = 0; i < 5; i++) {       // Я беру первые 5 значений
        cout << data[i] << " ";         // Я печатаю значение
    }                                   // Я заканчиваю цикл вывода
    cout << endl;                       // Я делаю перенос строки

    return 0;                           // Я завершаю программу
}                                       // Я закрываю main






Overwriting cpu_openmp.cpp


In [35]:
!g++ cpu_openmp.cpp -fopenmp -O2 -o cpu_openmp
!./cpu_openmp


CPU OpenMP time: 0.500845 ms
CPU check (first 5): 2 2 2 2 2 


In [36]:
%%writefile gpu_cuda.cu
#include <iostream>                     // Я подключаю ввод-вывод
#include <cuda_runtime.h>               // Я подключаю CUDA runtime
#include <chrono>                       // Я подключаю chrono для замера времени

using namespace std;                    // Я использую стандартное пространство имён

__global__ void multiplyByTwo(float* data, int n) {                 // Я создаю CUDA-ядро
    int i = blockIdx.x * blockDim.x + threadIdx.x;                  // Я вычисляю глобальный индекс
    if (i < n) {                                                    // Я проверяю, что индекс в пределах массива
        data[i] *= 2.0f;                                            // Я умножаю элемент на 2
    }                                                               // Я закрываю if
}                                                                   // Я заканчиваю ядро

int main() {                                                        // Я начинаю main
    const int N = 1000000;                                          // Я задаю размер массива
    size_t bytes = N * sizeof(float);                               // Я считаю размер массива в байтах

    float* h_data = new float[N];                                   // Я создаю массив на CPU
    for (int i = 0; i < N; i++) h_data[i] = 1.0f;                   // Я заполняю массив единицами

    float* d_data = nullptr;                                        // Я объявляю указатель на GPU
    cudaMalloc(&d_data, bytes);                                     // Я выделяю память на GPU
    cudaMemcpy(d_data, h_data, bytes, cudaMemcpyHostToDevice);      // Я копирую массив на GPU

    int threads = 256;                                              // Я задаю количество потоков в блоке
    int blocks = (N + threads - 1) / threads;                       // Я вычисляю количество блоков

    cudaDeviceSynchronize();                                        // Я синхронизирую GPU перед замером

    auto start = chrono::high_resolution_clock::now();              // Я фиксирую старт времени
    multiplyByTwo<<<blocks, threads>>>(d_data, N);                  // Я запускаю ядро на GPU
    cudaDeviceSynchronize();                                        // Я жду завершения вычислений
    auto end = chrono::high_resolution_clock::now();                // Я фиксирую конец времени

    cudaMemcpy(h_data, d_data, bytes, cudaMemcpyDeviceToHost);      // Я копирую результат обратно на CPU

    chrono::duration<double, milli> ms = end - start;               // Я считаю время выполнения
    cout << "GPU CUDA kernel time: " << ms.count() << " ms" << endl; // Я вывожу время

    cout << "GPU check (first 5): ";                                // Я вывожу проверку первых 5 элементов
    for (int i = 0; i < 5; i++) cout << h_data[i] << " ";           // Я печатаю первые 5 значений
    cout << endl;                                                   // Я делаю перенос строки

    cudaFree(d_data);                                               // Я освобождаю память GPU
    delete[] h_data;                                                // Я освобождаю память CPU

    return 0;                                                       // Я завершаю программу
}                                                                   // Я закрываю main





Writing gpu_cuda.cu


In [37]:
!nvcc gpu_cuda.cu -o gpu_cuda -gencode arch=compute_75,code=sm_75
!./gpu_cuda


GPU CUDA kernel time: 0.128031 ms
GPU check (first 5): 2 2 2 2 2 


In [38]:
%%writefile hybrid.cu
#include <iostream>                     // Я подключаю ввод-вывод
#include <vector>                       // Я подключаю vector
#include <omp.h>                        // Я подключаю OpenMP
#include <cuda_runtime.h>               // Я подключаю CUDA runtime
#include <chrono>                       // Я подключаю chrono для замера времени

using namespace std;                    // Я использую стандартное пространство имён

__global__ void multiplyByTwo(float* data, int n) {                 // Я создаю CUDA-ядро
    int i = blockIdx.x * blockDim.x + threadIdx.x;                  // Я считаю глобальный индекс
    if (i < n) {                                                    // Я проверяю границы
        data[i] *= 2.0f;                                            // Я умножаю элемент на 2
    }                                                               // Я закрываю if
}                                                                   // Я заканчиваю ядро

int main() {                                                        // Я начинаю main
    const int N = 1000000;                                          // Я задаю размер массива
    const int half = N / 2;                                         // Я делю массив пополам
    size_t halfBytes = half * sizeof(float);                        // Я считаю байты для половины

    vector<float> data(N, 1.0f);                                    // Я создаю массив на CPU и заполняю единицами

    float* d_half = nullptr;                                        // Я объявляю указатель на GPU для второй половины
    cudaMalloc(&d_half, halfBytes);                                 // Я выделяю память на GPU под вторую половину
    cudaMemcpy(d_half, data.data() + half, halfBytes, cudaMemcpyHostToDevice); // Я копирую вторую половину на GPU

    int threads = 256;                                              // Я задаю потоки
    int blocks = (half + threads - 1) / threads;                    // Я считаю блоки для половины

    cudaDeviceSynchronize();                                        // Я синхронизирую GPU перед замером
    auto start = chrono::high_resolution_clock::now();              // Я фиксирую старт времени

    #pragma omp parallel sections                                   // Я запускаю параллельные секции
    {                                                               // Я открываю блок секций

        #pragma omp section                                         // Секция 1: CPU
        {                                                           // Я начинаю CPU-секцию
            for (int i = 0; i < half; i++) {                        // Я прохожусь по первой половине
                data[i] *= 2.0f;                                    // Я умножаю на 2 на CPU
            }                                                       // Я закрываю цикл
        }                                                           // Я заканчиваю CPU-секцию

        #pragma omp section                                         // Секция 2: GPU
        {                                                           // Я начинаю GPU-секцию
            multiplyByTwo<<<blocks, threads>>>(d_half, half);        // Я запускаю умножение на GPU
            cudaDeviceSynchronize();                                // Я жду завершения GPU
        }                                                           // Я заканчиваю GPU-секцию

    }                                                               // Я закрываю секции

    cudaMemcpy(data.data() + half, d_half, halfBytes, cudaMemcpyDeviceToHost); // Я копирую результат второй половины

    auto end = chrono::high_resolution_clock::now();                // Я фиксирую конец времени
    chrono::duration<double, milli> ms = end - start;               // Я считаю общее время гибридного выполнения

    cout << "Hybrid CPU+GPU time: " << ms.count() << " ms" << endl; // Я вывожу время

    cout << "Hybrid check (first 5): ";                             // Я проверяю первые 5 значений
    for (int i = 0; i < 5; i++) cout << data[i] << " ";             // Я вывожу первую часть
    cout << endl;                                                   // Я перенос строки

    cout << "Hybrid check (middle 5): ";                            // Я проверяю значения из второй половины
    for (int i = half; i < half + 5; i++) cout << data[i] << " ";   // Я вывожу элементы из GPU части
    cout << endl;                                                   // Я перенос строки

    cudaFree(d_half);                                               // Я освобождаю память GPU

    return 0;                                                       // Я завершаю программу
}                                                                   // Я закрываю main


Writing hybrid.cu


In [39]:
!nvcc hybrid.cu -Xcompiler -fopenmp -O2 -o hybrid -gencode arch=compute_75,code=sm_75
!./hybrid


Hybrid CPU+GPU time: 3.84044 ms
Hybrid check (first 5): 2 2 2 2 2 
Hybrid check (middle 5): 2 2 2 2 2 


**Вывод**

В данной работе я реализовала обработку массива тремя способами: на CPU с OpenMP, на GPU с CUDA и в гибридном режиме. CPU вариант хорошо показывает ускорение за счёт потоков, GPU вариант эффективен для массовых параллельных операций, а гибридный подход позволяет распределить нагрузку между CPU и GPU. Наибольший выигрыш гибридный режим даёт на больших массивах при правильном делении работы и минимизации копирования данных между CPU и GPU.

